In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


In [38]:
import matplotlib.pyplot as plt  # 数据可视化
import seaborn as sns        # 高级可视化工具
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV  # 数据集划分和超参数搜索
from sklearn.preprocessing import StandardScaler, OrdinalEncoder    # 数据预处理
from sklearn.compose import ColumnTransformer      # 多列不同预处理组合
from sklearn.pipeline import Pipeline              # 流水线化处理流程
from sklearn.ensemble import RandomForestClassifier  # 随机森林分类模型
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import (
    mean_squared_error,          # 均方误差（MSE），越小越好
    r2_score                     # R²分数，越接近1越好
)

In [39]:
train_data=pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
test_data=pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')
a = preprocessing.LabelEncoder()
train_data['Sex']=a.fit_transform(train_data['Sex'])
test_data['Sex']=a.fit_transform(test_data['Sex'])

In [40]:
train_X=train_data.drop(['id','Height','Body_Temp','Calories'],axis=1)
test_X=test_data.drop(['id','Height','Body_Temp'],axis=1)
train_y=train_data.loc[:,'Calories']


In [41]:
    X_tr, X_val, y_tr, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)
    scaler = StandardScaler()               # 初始化标准化器
    X_tr_scaled = scaler.fit_transform(X_tr)  # 在训练集上拟合标准化参数并转换
    X_val_scaled = scaler.transform(X_val)  # 验证集转换（使用训练集的参数）
    X_test_scaled = scaler.transform(test_X)  # 测试集转换

In [42]:
    models = [
        ('LinearRegression', LinearRegression()),  # 线性回归（无正则化）
        ('Ridge', Ridge(alpha=1.0)),                # 岭回归（alpha=1.0）
        ('Lasso', Lasso(alpha=0.1)),                # Lasso回归（alpha=0.1）
        ('RandomForest', RandomForestRegressor(n_estimators=200, random_state=42)),  # 随机森林（200棵树）
        ('XGBoost', XGBRegressor(n_estimators=200, random_state=42, verbosity=0)),
        ('LightGBM', LGBMRegressor(n_estimators=200, random_state=42))
    ]


In [43]:
best_sc=np.inf
best_model=None
best_name=""
res=[]

In [44]:
for name, model in models:
    # 训练模型（线性模型用标准化数据，树模型可用原始数据，此处统一用标准化数据）
    if name in ['LinearRegression', 'Ridge', 'Lasso']:
        model.fit(X_tr_scaled,y_tr)
        val_pred = model.predict(X_val_scaled)  # 线性模型预测（可能负数）
    else:
        model.fit(X_tr_scaled, y_tr)  # 树模型无需标准化（可选）
        val_pred = model.predict(X_val_scaled)  # 树模型预测（通常非负）
    # 关键修正：强制预测值非负（处理线性模型的负数问题）
    val_pred = np.maximum(val_pred, 0)  # 负数置0
    msle = mean_squared_log_error(y_val, val_pred)
    rmsle = np.sqrt(msle)
    res.append((name,rmsle))                # 记录结果
    if rmsle < best_sc:                            # 更新最优模型
        best_sc =rmsle 
        best_model = model
        best_name = name
    print(f"\n最优模型：{best_name}，验证集rmsle: {best_sc:.4f}")  # 输出最优模型



最优模型：LinearRegression，验证集rmsle: 0.6452

最优模型：Ridge，验证集rmsle: 0.6452

最优模型：Lasso，验证集rmsle: 0.6414

最优模型：RandomForest，验证集rmsle: 0.0660

最优模型：XGBoost，验证集rmsle: 0.0629
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006409 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 236
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 5
[LightGBM] [Info] Start training from score 88.298465

最优模型：XGBoost，验证集rmsle: 0.0629


In [45]:
    test_pred = best_model.predict(X_test_scaled)       
    res_df = pd.DataFrame({'id':test_data['id'], 'Calories': test_pred})
    res_df = res_df.head(250000)
    res_df.to_csv('/kaggle/working/aac.csv', index=False)      